In [1]:
import ipywidgets as widgets
from geometry_msgs.msg import Twist
from matplotlib import animation, rc
from nav_msgs.msg import Odometry
import rospy
import jupyros as jr
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from rt2_assignment1.srv import Command

%matplotlib widget

#define client
rospy.init_node('better_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)

 

button1 = widgets.Button(description='Start')
button2 = widgets.Button(description='Stop')
#HBox([button1, button2])
out = widgets.Output()

def on_button1_clicked(_):
    # "linking function with output"
    with out:
        # what happens when we press the button
        #clear_output()
        ui_client("start")
        print('start random moving...')
        
def on_button2_clicked(_):
    # "linking function with output"
    with out:
        # what happens when we press the button
        #clear_output()
        ui_client("stop")
        print('Stop random moving')

# linking button and function together using a button's method
button1.on_click(on_button1_clicked)
# displaying button and its output together
#widgets.HBox([button1,button2, out])

# linking button and function together using a button's method
button2.on_click(on_button2_clicked)
# displaying button and its output together
#widgets.HBox([button2, out])
widgets.VBox([HBox([button1,button2]), out])

#out





In [2]:
#pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from matplotlib import animation, rc
import ipywidgets as widgets
#msg = Twist()
#initialization


linear = widgets.FloatSlider(value=0.3, min=0.1, max=0.6)
a1 = widgets.HBox([widgets.Label(value="Linear Velocity"), linear])
display(a1)

angular = widgets.FloatSlider(value=0.3, min=0.1, max=0.6)
a2 = widgets.HBox([widgets.Label(value="Angular Velocity"), angular])
display(a2)

kp = widgets.FloatSlider(value=3, min=0.1, max=5.0)
a3 = widgets.HBox([widgets.Label(value="Kp_a"), kp])
display(a3)



def linear_value_change(change):
    #global msg
    #msg.linear.x = change['new']
    l_vel = change['new']
    rospy.set_param("linear_vel",l_vel)
    #pub.publish(msg)
    
def angular_value_change(change):
    #global msg
    #msg.angular.z = change['new']
    a_vel = change['new']
    rospy.set_param("angular_vel",a_vel)    
    #pub.publish(msg)

def kp_value_change(change):
    #global msg
    #msg.angular.z = change['new']
    kp_vel = change['new']
    rospy.set_param("kp_vel",kp_vel)    
    #pub.publish(msg)
    
linear.observe(linear_value_change, names='value')
angular.observe(angular_value_change, names='value')
kp.observe(kp_value_change, names='value')
